This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [152]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.regularizers import l2
import numpy as np

In [153]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
# MUFFIN VS CHIHUAHUA
train_dir = "mvc/train"
test_dir = "mvc/test"

# CUSTOM DATASET
# train_dir = "mvc/train"
# test_dir = "mvc/test"

In [154]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [155]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [156]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting
model = models.Sequential([
    layers.Conv2D(
        32, (3, 3),
        activation='relu',
        input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
        kernel_regularizer=l2(0.1)
        ),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.3),

    layers.Conv2D(
        64, (3, 3),
        activation='relu',
        kernel_regularizer=l2(0.1)
        ),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.3),  

    layers.Conv2D(
        128, (3, 3),
        activation='relu',
        kernel_regularizer=l2(0.1)
        ),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    
    layers.Dense(1, activation='sigmoid')
])

In [157]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [158]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 112s 919ms/step - accuracy: 0.5438 - loss: 3.6378 - val_accuracy: 0.4804 - val_loss: 0.8864
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 114s 955ms/step - accuracy: 0.5876 - loss: 0.7687 - val_accuracy: 0.4603 - val_loss: 0.7590
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 204s 1s/step - accuracy: 0.6175 - loss: 0.6993 - val_accuracy: 0.6571 - val_loss: 0.6888
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 134s 1s/step - accuracy: 0.6550 - loss: 0.6662 - val_accuracy: 0.6423 - val_loss: 0.6762
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 120s 1s/step - accuracy: 0.6911 - loss: 0.6369 - val_accuracy: 0.7577 - val_loss: 0.5609
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - accuracy: 0.7117 - loss: 0.6057 - val_accuracy: 0.7810 - val_loss: 0.5301
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 115s 968ms/step - accuracy: 0.7284 - loss: 0.5918 - val_accuracy: 0.7344 - val_loss: 0.5678
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 108s 903ms/step - accuracy: 0.7326 - loss: 0.587

In [159]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

c:\Users\USER\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


37/37 ━━━━━━━━━━━━━━━━━━━━ 16s 445ms/step - accuracy: 0.8007 - loss: 0.5271
Test Accuracy: 0.8006756901741028


In [160]:
# SAVE THE MODEL
model.save('exercise_6_trained_model_improved_baladjay_barlas.h5')

In [161]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_trained_model_improved_baladjay_barlas.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    print(f"Prediction: {label} (confidence: {pred:.2f})")


In [162]:
# Example usage:
predict_image("test/run_1.jpg")
predict_image("test/run_2.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step


Prediction: Chihuahua (confidence: 0.86)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
Prediction: Chihuahua (confidence: 0.64)


Custom Dataset: Cat vs Dog

In [163]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.

# CUSTOM DATASET
train_dir = "catvdog/train"
test_dir = "catvdog/test"

In [164]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [165]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [183]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting
model = models.Sequential([
    layers.Conv2D(
        32, (3, 3),
        activation='relu',
        input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
        kernel_regularizer=l2(0.001)
        ),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(
        64, (3, 3),
        activation='relu',
        kernel_regularizer=l2(0.001)
        ),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),  

    layers.Conv2D(
        128, (3, 3),
        activation='relu',
        kernel_regularizer=l2(0.001)
        ),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    
    layers.Dense(1, activation='sigmoid')
])

In [184]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [185]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 201s 989ms/step - accuracy: 0.4967 - loss: 0.7823 - val_accuracy: 0.4997 - val_loss: 0.7400
Epoch 2/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 120s 593ms/step - accuracy: 0.5225 - loss: 0.7273 - val_accuracy: 0.5703 - val_loss: 0.7141
Epoch 3/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 135s 671ms/step - accuracy: 0.5540 - loss: 0.7065 - val_accuracy: 0.5809 - val_loss: 0.7011
Epoch 4/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 141s 699ms/step - accuracy: 0.5737 - loss: 0.6951 - val_accuracy: 0.6084 - val_loss: 0.6808
Epoch 5/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 117s 581ms/step - accuracy: 0.6271 - loss: 0.6648 - val_accuracy: 0.6440 - val_loss: 0.6602
Epoch 6/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 119s 591ms/step - accuracy: 0.6401 - loss: 0.6454 - val_accuracy: 0.6415 - val_loss: 0.6258
Epoch 7/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 119s 594ms/step - accuracy: 0.6512 - loss: 0.6419 - val_accuracy: 0.6602 - val_loss: 0.6296
Epoch 8/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 119s 590ms/step - accuracy: 0.6757 -

In [186]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 31s 490ms/step - accuracy: 0.7054 - loss: 0.5869
Test Accuracy: 0.7053880095481873


In [187]:
# SAVE THE MODEL
model.save('exercise_6_custom_baladjay_barlas.h5')

In [188]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_custom_baladjay_barlas.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Cat" if pred >= 0.5 else "Dog"
    print(f"Prediction: {label} (confidence: {pred:.2f})")


In [190]:
# Example usage:
predict_image("test/run_3.jpg")
predict_image("test/run_4.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step


Prediction: Cat (confidence: 0.57)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
Prediction: Dog (confidence: 0.31)
